# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/datasets/camnugent/california-housing-prices

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает медианную стоимость домовладения на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [ ]:
import numpy as np
import torch, csv

def gen_ser(lst):
    # Создадим итератор при помощи генератора.
    for item in lst:
        yield item

In [ ]:
import pandas as pd

In [ ]:
#Создадим класс датасета:

class RangeDataset(torch.utils.data.Dataset):
    def __init__(self, file, start, end, step=1): #нумерация с 1
        self.start = start
        self.end = end
        self.step = step
    
        self.file = file
        self.len_csv = 0
        self.gen = gen_ser([x for x in range(start, end+1, step)]) #все нужные нам индексы
        self.header = None #шапка
        self.cur_ind = 0 #текущий номер строки - используется только при чтении
        
        #создаем текущий reader csv-файла
        csvfile = open(self.file, newline='')
        self.reader = csv.reader(csvfile, delimiter=',') #НАСТРОЙКИ!
        self.df = pd.read_csv(csvfile)
        #csvfile.close()

    def __len__(self):
      return self.df.shape[0]
      '''
        if self.len_csv == 0:
            i = 0
            while True:
                try:
                    if i==0: #первая строка
                        self.header = self.reader.__next__() #записываем шапку
                    else:
                        self.reader.__next__()
                    
                    i+=1
                except:
                    break
                
            self.len_csv = i - 1 #не учитываем "шапку"
            
            #заново создаем текущий reader csv-файла
            csvfile = open(self.file, newline='')
            self.reader = csv.reader(csvfile, delimiter=',') 
            #csvfile.close()
            
        else:
            pass
        
        return self.len_csv
        '''

    def __getitem__(self, indx):
        '''
        value = self.reader.__next__()
        self.cur_ind += 1
        while self.cur_ind < indx:

            try:
                value = self.reader.__next__()
                self.cur_ind+=1

            except:                
                #заново создаем текущий reader csv-файла
                csvfile = open(self.file, newline='')
                self.reader = csv.reader(csvfile, delimiter=',') 
                
                print('RangeDataset.__getitem__: ПРЕДУПРЕЖДЕНИЕ: достигнут конец файла')
                
                return
        '''
        value = self.df.iloc[indx,:].values        
        return value

In [ ]:
dataset = RangeDataset('/content/sample_data/california_housing_train.csv', 1, 500, 2)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2, drop_last=True)

for i, batch in enumerate(data_loader):
    print(i, batch)

Выходные данные были обрезаны до нескольких последних строк (5000).
        [-1.2244e+02,  3.7790e+01,  5.2000e+01,  1.9030e+03,  4.6100e+02,
          8.3100e+02,  4.3300e+02,  4.4464e+00,  5.0000e+05],
        [-1.1828e+02,  3.4060e+01,  1.4000e+01,  1.7870e+03,  8.5300e+02,
          2.2510e+03,  7.6300e+02,  1.1642e+00,  4.0000e+05]],
       dtype=torch.float64)
3695 tensor([[-1.2206e+02,  3.7390e+01,  2.6000e+01,  1.8000e+01,  4.0000e+00,
          8.0000e+00,  4.0000e+00,  3.7500e+00,  3.7500e+05],
        [-1.2245e+02,  3.7780e+01,  5.2000e+01,  1.3450e+03,  2.9100e+02,
          5.6000e+02,  2.9400e+02,  3.7159e+00,  4.9440e+05],
        [-1.2228e+02,  3.7810e+01,  5.2000e+01,  3.8600e+02,  1.6400e+02,
          3.4600e+02,  1.5500e+02,  8.0750e-01,  1.3750e+05],
        [-1.1762e+02,  3.4130e+01,  2.0000e+01,  3.2160e+03,  5.1600e+02,
          1.6550e+03,  5.2400e+02,  5.1261e+00,  1.5880e+05]],
       dtype=torch.float64)
3696 tensor([[-1.1770e+02,  3.4060e+01,  7.0000e+00, 

Ничего не получается, поэтому загружу датасет привычным способом: с помощью Pandas

In [68]:
import pandas as pd

data = pd.read_csv('/content/sample_data/california_housing_train.csv', delimiter=',')
print(len(data))

17000


In [69]:
data.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
5,-114.58,33.63,29.0,1387.0,236.0,671.0,239.0,3.3438,74000.0
6,-114.58,33.61,25.0,2907.0,680.0,1841.0,633.0,2.6768,82400.0
7,-114.59,34.83,41.0,812.0,168.0,375.0,158.0,1.7083,48500.0
8,-114.59,33.61,34.0,4789.0,1175.0,3134.0,1056.0,2.1782,58400.0
9,-114.60,34.83,46.0,1497.0,309.0,787.0,271.0,2.1908,48100.0


Проверим датафрейм на nan-значения:

In [70]:
data.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

В поле total_bedrooms имеется 207 пропусков. ЗАполним их медианным значением:

In [71]:
med = data.total_bedrooms.median()
data.total_bedrooms.fillna(med,inplace=True)

Смотрим результаты теперь:

In [72]:
data.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

Обратотаем поля следующим образом: вещественные - с помощью StandardScaler, категориальный признак ocean_proximity - с помощью OHE. Предсказывать будем median_house_value - медианную стоимость дома

In [73]:
from sklearn.preprocessing import StandardScaler

Z_longitude = StandardScaler()
Z_latitude = StandardScaler()
Z_housing_median_age = StandardScaler()
Z_total_rooms = StandardScaler()
Z_total_bedrooms = StandardScaler()
Z_population = StandardScaler()
Z_households = StandardScaler()
Z_median_income = StandardScaler()

#обработаем вещественные признаки
data.longitude = Z_longitude.fit_transform(np.expand_dims(data.longitude.to_numpy(),axis=1))
data.latitude = Z_latitude.fit_transform(np.expand_dims(data.latitude.to_numpy(),axis=1))
data.housing_median_age = Z_housing_median_age.fit_transform(np.expand_dims(data.housing_median_age.to_numpy(),axis=1))
data.total_rooms = Z_total_rooms.fit_transform(np.expand_dims(data.total_rooms.to_numpy(),axis=1))
data.total_bedrooms = Z_total_bedrooms.fit_transform(np.expand_dims(data.total_bedrooms.to_numpy(),axis=1))
data.population = Z_population.fit_transform(np.expand_dims(data.population.to_numpy(),axis=1))
data.households = Z_households.fit_transform(np.expand_dims(data.households.to_numpy(),axis=1))
data.median_income = Z_median_income.fit_transform(np.expand_dims(data.median_income.to_numpy(),axis=1))

#обработаем категориальные признаки и перемешаем данные
data = pd.get_dummies(data,columns=['ocean_proximity'])
data = data.sample(frac=1)

#смотрим итоговый датафрейм
data.head(10)

KeyError: ignored

Делим выборку на обучающую и тестовую

In [80]:
from sklearn.model_selection import train_test_split

X = data.drop('median_house_value',axis=1).to_numpy()
y = np.expand_dims(data.median_house_value.to_numpy(), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(12750, 8) (4250, 8) (12750, 1) (4250, 1)


In [81]:
X.max()

29.841307855883887

Переводим все массивы в тензоры

In [82]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

Создадим 6-слойный перцептрон

In [83]:
from tqdm import tqdm

import torch.nn.functional as fnc, torch.nn as nn
import torch.optim as opt

#6-слойный перцептрон c ФА ReLU
class Model_6l(nn.Module):
    def __init__(self,lays,optim):
        super(Model_6l, self).__init__()
        self.bn1 = nn.BatchNorm1d(lays[0])
        self.lay12=nn.Linear(lays[0],lays[1]).to(device)
        self.bn2 = nn.BatchNorm1d(lays[1])
        self.dp2 = nn.Dropout(0.2)
        self.lay23=nn.Linear(lays[1],lays[2]).to(device)
        self.bn3 = nn.BatchNorm1d(lays[2])
        self.dp3 = nn.Dropout(0.2)
        self.lay34=nn.Linear(lays[2],lays[3]).to(device)
        self.bn4 = nn.BatchNorm1d(lays[3])
        self.lay45=nn.Linear(lays[3],lays[4]).to(device)
        self.bn5 = nn.BatchNorm1d(lays[4])
        self.lay56=nn.Linear(lays[4],lays[5]).to(device)
        
        #if optim == 'SGD': #оптимизаторы
        #    self.opt=opt.SGD(self.parameters(), lr=0.00000001, momentum=0.9) 
        #elif optim == 'Adam':
        #    self.opt=opt.Adam(self.parameters(), lr=0.00000001, betas=(0.9, 0.999))
        #elif optim == 'RMSprop':
        #    self.opt=opt.RMSprop(self.parameters(), lr=0.00000001, momentum=0.9)
        #
        #self.criterion=nn.MSELoss() #функция потерь для задачи регрессии

    def forward(self, x):
        x = self.bn1(x)
        x = self.lay12(x)
        x = fnc.relu(x)
        x = self.bn2(x)
        x = self.dp2(x)
        x = self.lay23(x)
        x = fnc.relu(x)
        x = self.bn3(x)
        x = self.dp3(x)
        x = self.lay34(x)
        x = fnc.relu(x)
        x = self.bn4(x)
        x = self.lay45(x)
        x = fnc.relu(x)
        x = self.bn5(x)
        x = self.lay56(x)
        
        return fnc.relu(x)

In [84]:
#подготовка обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

size_nn = [8, 256, 160, 80, 48, 1] #зафиксируем размер сети
optims = ['SGD', 'Adam', 'RMSprop']

errs_6l = []
mdls_6l = []

Будем обучать каждую сеть на протяжении 1000 эпох

In [87]:
#определим размер пакета
batch_size = 64*16
smpls_train = len(y_train)
batches = int(np.ceil(smpls_train/batch_size))

X_train_mean  = X_train.mean(axis = 1)
X_train_std  = X_train.std(axis = 1)
for k in range(8):
              X_train[:,k]=(X_train[:,k]- X_train_mean[k])/X_train_std[k]
for k in range(8):
              X_test[:,k]=(X_test[:,k]- X_train_mean[k])/X_train_std[k]
y_test =( y_test - y_train.mean())/y_train.std()
y_train =( y_train - y_train.mean())/y_train.std()
for optim in optims:
    net = Model_6l(size_nn, optim)
    opt = torch.optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.999))
    criterion=nn.MSELoss()
    
    for epoch in tqdm(range(10)):  
        running_loss = 0.0
        for i in range(batches):
            if i+1 != batches:
                inputs = X_train[i*batch_size:(i+1)*batch_size, :]
                labels = y_train[i*batch_size:(i+1)*batch_size, :]

            else:
                inputs1 = X_train[i*batch_size:smpls_train, :]
                labels1 = y_train[i*batch_size:smpls_train, :]
                
                resid = batch_size - (smpls_train - i*batch_size)
                    
                inputs2 = X_train[0:resid, :]
                labels2 = y_train[0:resid, :]
                    
                inputs = torch.cat((inputs1,inputs2))
                labels = torch.cat((labels1,labels2))
                
                del inputs1, labels1, inputs2, labels2
            #print(labels[:5])
            # обнуляем градиент
            opt.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            #print(net.lay12.weight)
            #net.
            opt.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
        if epoch != 0:    # печатаем каждые 100 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss/batches )) # откуда 2000?
                running_loss = 0.0
                
            
        
        #вычисление ошибки на тестовом наборе данных
        
        y_pred = net(X_test)
        loss_test = criterion(y_pred, y_test).item()
        
        errs_6l.append(loss_test)
        mdls_6l.append(net)
        print(loss_test)

 10%|█         | 1/10 [00:00<00:03,  2.49it/s]

0.7316602468490601


 20%|██        | 2/10 [00:00<00:03,  2.50it/s]

[2,    13] loss: 0.698
0.6789954304695129


 30%|███       | 3/10 [00:01<00:02,  2.49it/s]

...

[8,    13] loss: 0.594
0.5937904715538025


 90%|█████████ | 9/10 [00:02<00:00,  3.76it/s]

[9,    13] loss: 0.585
0.5808684229850769


100%|██████████| 10/10 [00:02<00:00,  3.74it/s]

[10,    13] loss: 0.579
0.5814640522003174
